### To speed up the really common HS calculation, we are going to train a neural network to do it. 

In [80]:
from fast_utils.hand_strength.original_HS import *
from fast_utils.hand_strength.nn_HS import encode_hs
from fast_utils.expected_hand_strength.nn_EHS import *

from sklearn.model_selection import train_test_split
from pickle import dump, load
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model
import numpy as np
from random import choice
from copy import deepcopy
from deuces.deck import Deck
from deuces.evaluator import Evaluator


In [3]:
hs_model = load_model('fast_utils/hand_strength/HS_model.h5')

In [4]:
try: 
    encodings = load(open('fast_utils/expected_hand_strength/original_EHS_training_x.pickle','rb'))
    EHSs = load(open('fast_utils/expected_hand_strength/original_EHS_training_y.pickle','rb'))
except FileNotFoundError:
    d = Deck()
    en = EvaluatorN()

    encodings = []
    EHSs = []

    for board_i in range(150000):
        d.shuffle()
        our_hand = d.draw(2)
        board = d.draw(choice([3,4]))
        encodings.append(encode_ehs(our_hand, board, en, hs_model))
        EHSs.append(original_EHS(our_hand, board, en, hs_model, 100))
        if board_i % 100 == 0:
            print(board_i)
    dump(encodings, open('fast_utils/expected_hand_strength/original_EHS_training_x.pickle','wb'))
    dump(HSs, open('fast_utils/expected_hand_strength/original_EHS_training_y.pickle','wb'))


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

KeyboardInterrupt: 

In [106]:
x_train, x_test, y_train, y_test = train_test_split(np.asarray(encodings[:-1]), np.asarray(EHSs)[:,0], test_size=0.05)

In [107]:
model = Sequential()
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10)
model.save("fast_utils/expected_hand_strength/EHS_model.h5")

Train on 41792 samples, validate on 2200 samples
Epoch 1/10
40896/41792 [============================>.] - ETA: 0s - loss: 0.0079 - mean_absolute_error: 0.06241792/41792 [==============================] - 2s 41us/step - loss: 0.0077 - mean_absolute_error: 0.0621 - val_loss: 0.0032 - val_mean_absolute_error: 0.0426
Epoch 2/10
41792/41792 [==============================] - 1s 33us/step - loss: 0.0027 - mean_absolute_error: 0.0403 - val_loss: 0.0029 - val_mean_absolute_error: 0.0437
Epoch 3/10
41792/41792 [==============================] - 2s 37us/step - loss: 0.0020 - mean_absolute_error: 0.0348 - val_loss: 0.0019 - val_mean_absolute_error: 0.0336
Epoch 4/10
41792/41792 [==============================] - 2s 36us/step - loss: 0.0017 - mean_absolute_error: 0.0315 - val_loss: 0.0015 - val_mean_absolute_error: 0.0292
Epoch 5/10
41792/41792 [==============================] - 1s 34us/step - loss: 0.0015 - mean_absolute_error: 0.0296 - val_loss: 0.0014 - val_mean_absolute_error: 0.0290
Epoch 6/

In [221]:
d = Deck()
ours = d.draw(2)
board = d.draw(3)
print(nn_EHS(ours, board, en, hs_model, model))
print(original_EHS(ours, board, en, hs_model))

0.34216905
(0.31756338, 0.16721343)
